In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as pltn
import os
from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram
from os import path
from dipy.tracking.local import ParticleFilteringTracking
from dipy.core.gradients import gradient_table, GradientTable
from dipy.data import get_sphere, Sphere, HemiSphere
from dipy.direction import (peaks_from_model, ProbabilisticDirectionGetter)
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.io.streamline import save_trk, load_trk
from dipy.reconst.peaks import reshape_peaks_for_visualization
from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,auto_response, recursive_response)
from dipy.segment.mask import median_otsu
from dipy.tracking.local import (LocalTracking, ThresholdTissueClassifier, ActTissueClassifier, CmcTissueClassifier)
from dipy.tracking.streamline import Streamlines
from dipy.tracking.utils import random_seeds_from_mask, move_streamlines
from dipy.viz import actor, window
from dipy.tracking.utils import target, density_map
from dipy.reconst.shm import sh_to_sf
from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.io import read_bvals_bvecs
from dipy.io.image import save_nifti
from dipy.reconst.dti import fractional_anisotropy, color_fa, lower_triangular
from dipy.reconst.dti import TensorModel
from dipy.viz import window, actor
import dipy.reconst.dti as dti
from dipy.viz import window, actor

In [2]:
os.chdir('/space/hemera/1/users/cmaffei/ex_vivo/')
wd = '/space/hemera/1/users/cmaffei/ex_vivo/'

In [3]:
# # Import DWI data, bvec, bval, and binary mask
# fimg =  ('dwi.nii')
# img = nib.load(fimg)
# data = img.get_data()
# print("Data Shape: " + str(data.shape))
# affine = img.affine #no need to store the affine tho. 

#Bval and bvec file information import
fbval= ('hemi/bvals')
fbvec = ("hemi/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
#Read the voxel size from the image header.
voxel_size = img.header.get_zooms()[:3]
print ('Voxel Size: ' + str(voxel_size))

IOError: [Errno 2] No such file or directory: 'hemi/bvals'

In [ ]:
#Fit the tensor model to data
tenmodel = dti.TensorModel(gtab)
tenfit = tenmodel.fit(data)

In [ ]:
from dipy.data import get_sphere
sphere = get_sphere('symmetric724')

from dipy.viz import window, actor

# Enables/disables interactive visualization
interactive = True

ren = window.Renderer()

evals = tenfit.evals[:, :, 111:112]
evecs = tenfit.evecs[:, :, 111:112]
FA = fractional_anisotropy(tenfit.evals)
RGB = color_fa(FA, tenfit.evecs)
cfa = RGB[:, :, 111:112]
cfa /= cfa.max()
tensor_actor = actor.tensor_slicer(evals, evecs, scalar_colors=cfa, sphere=sphere, scale=0.3)
ren.add(tensor_actor)
if interactive:
    window.show(ren)

In [ ]:
#recursive response
FA = nib.load('dti_fa.nii').get_data()
# MD = mean_diffusivity(tenfit.evals)
wm_mask = FA >= 0.5
nib.save(nib.Nifti1Image(wm_mask.astype('float32'), img.affine, img.header), 'wm.nii.gz')

response = recursive_response(gtab, data, mask=wm_mask, sh_order=6,
                              peak_thr=0.01, init_fa=0.2,
                              init_trace=0.0021, iter=20, convergence=0.0011, 
                              parallel=True, nbr_processes = 4)

In [ ]:
#check the shape of the signal of the response function, which should be like a pancake:
sphere = get_sphere('symmetric724')
response_signal = response.on_sphere(sphere)
# transform our data from 1D to 4D
response_signal = response_signal[None, None, None, :]
response_actor = actor.odf_slicer(
    response_signal, sphere=sphere, colormap='plasma')
interactive = True
ren = window.Renderer()

ren.add(response_actor)
print('Saving illustration as csd_recursive_response.png')
window.record(ren, out_path='csd_recursive_response.png', size=(200, 200))
if interactive:
    window.show(ren)

In [ ]:
#set sub
sub='I43'

# Import DWI data, bvec, bval, and binary mask
fimg =  (str(sub)+'/dwi.nii.gz')
img = nib.load(fimg)
data = img.get_data()
print("Data Shape: " + str(data.shape))
affine = img.affine #no need to store the affine tho. 

# # #Bval and bvec file information import
fbval= (str(sub)+'/bvals')
fbvec = (str(sub)+"/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
#Read the voxel size from the image header.
voxel_size = img.header.get_zooms()[:3]
print ('Voxel Size: ' + str(voxel_size))

#mask
fimg =  (str(sub)+'/mask.nii.gz')
img = nib.load(fimg)
mask = img.get_data()

data_small=data[104:172, 150:200, 78:84]
nib.save(nib.Nifti1Image(data_small.astype('float32'), img.affine, img.header), 'data_small.nii.gz')

#flipping bvecs
bvecs[:,0] *= -1
gtab = gradient_table(bvals,bvecs)

#mask
fimg =  (str(sub)+'/cc_mask.nii')
img = nib.load(fimg)
wm_mask = img.get_data()
wm_mask= wm_mask > 0.01
sphere = get_sphere('symmetric724')


fa_init  = [0.05, 0.06, 0.1, 0.2]
peak_thr = [0.05, 0.1]

for init in fa_init:
        for pr in peak_thr:
            response = recursive_response(gtab, data, mask=wm_mask, sh_order=8,
                              peak_thr=pr, init_fa=init,
                              init_trace=0.0021, iter=20, convergence=0.001, 
                              parallel=True, nbr_processes=4)

            #check the shape of the signal of the response function, which should be like a pancake:
            response_signal = response.on_sphere(sphere)
            # transform our data from 1D to 4D
            response_signal = response_signal[None, None, None, :]
            response_actor = actor.odf_slicer(
            response_signal, sphere=sphere, colormap='plasma')
            interactive = False
            ren = window.Renderer()
            ren.add(response_actor)
            window.record(ren, out_path=str(sub)+'/rf_'+str(init)+'_'+str(pr)+'.png', size=(200, 200))
            if interactive:
                window.show(ren)
            #computing fods
            csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order=8)
            csd_peaks = peaks_from_model(model=csd_model,
                             data=data_small,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             return_odf=True, return_sh=True)
            #save odf
            nib.save(nib.Nifti1Image(csd_peaks.shm_coeff.astype(np.float32),
                                  img.affine, img.header), str(sub)+'/odfs_'+str(init)+'_'+str(pr)+'.nii.gz')

Data Shape: (270, 288, 176, 102)
Voxel Size: (0.75, 0.75, 0.75)


/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:588: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:588: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:588: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:588: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:1033: RuntimeWarning: invalid value encountered in true_divide
  single_peak_mask = (vals[:, 1] / vals[:, 0]

In [ ]:
data_small=data[10:96, 100:201, 100:174]
data_small.shape
first_vol = data_small[:, :, :, 0]
plt.imshow(first_vol[:, 36, :], cmap='gray')
plt.show()

In [ ]:
csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order=6)
sphere = get_sphere('symmetric724')
csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             return_odf=True, return_sh=True)


In [ ]:
#save odf
nib.save(nib.Nifti1Image(csd_peaks.shm_coeff.astype(np.float32),
                                  img.affine, img.header), 'odfs_recursive_csd.nii.gz')



In [ ]:
#loading odfs
sphere = get_sphere('symmetric724')
fimg = ("odfs_recursive_csd.nii.gz")
img = nib.load(fimg)
odfs = img.get_data()
odfs = sh_to_sf(odfs, sphere, sh_order=8)

In [ ]:
#Enables/disables interactive visualization
sphere = get_sphere('symmetric724')
ren = window.Renderer()
interactive = True
fodf_spheres = actor.odf_slicer(odfs, sphere=sphere, scale=0.6, norm=False, colormap='jet')
fodf_spheres.display_extent(70, 70, 0, 256, 0, 192)
seedroi_actor = actor.contour_from_roi(slf_mask,
                                        color=[0,1,1], opacity=0.3)

# ren.set_camera(position=(10, 0, 0))
ren.add(fodf_spheres)
ren.add(seedroi_actor)
print('Saving illustration as csd_odfs.png')
window.record(ren, out_path='csd_fods.png', size=(600, 600))
if interactive:
    window.show(ren)

In [ ]:
slf_mask=("slfI_approx_roi.nii")
img = nib.load(slf_mask)
slf_mask = img.get_data()

In [ ]:
tissue_classifier = ThresholdTissueClassifier(FA, 0.2)
seeds = random_seeds_from_mask(FA > 0.3, seeds_count=5)

In [ ]:
prob_dg = ProbabilisticDirectionGetter.from_shcoeff(csd_peaks.shm_coeff, max_angle=20, sphere=sphere)


In [ ]:
all_streamlines_act_classifier = LocalTracking(prob_dg,
                                               tissue_classifier,
                                               seeds, 
                                               affine=np.eye(4), step_size=0.35,
                                               return_all=True)
streamlines = Streamlines(all_streamlines_act_classifier)
save_trk("csd_recursive_prob_cmc_npv5_20_0.35.trk",
         streamlines,
         img.affine,
         shape=img.shape[:3], vox_size=img.header.get_zooms()[:3])

    ### Block Analysis

In [30]:
# Import DWI data, bvec, bval, and binary mask
fimg =  ('block_I35/dwi_las.nii')
img = nib.load(fimg)
data = img.get_data()
print("Data Shape: " + str(data.shape))
affine = img.affine #no need to store the affine tho. 

#Bval and bvec file information import
fbval= ('block_I35/bvals')
fbvec = ("block_I35/bvecs")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)
#Read the voxel size from the image header.
voxel_size = img.header.get_zooms()[:3]
print ('Voxel Size: ' + str(voxel_size))

Data Shape: (112, 96, 96, 251)
Voxel Size: (0.34999999, 0.34999999, 0.34999999)


In [4]:
sphere = get_sphere('symmetric724')

In [27]:
#Fit the tensor model to data
tenmodel = dti.TensorModel(gtab)
tenfit = tenmodel.fit(data)

In [32]:
from dipy.data import get_sphere
sphere = get_sphere('symmetric724')

# Enables/disables interactive visualization
interactive = True

# ren = window.Renderer()

# evals = tenfit.evals[:, :, 40:41]
# evecs = tenfit.evecs[:, :, 40:41]
FA = fractional_anisotropy(tenfit.evals)
save_nifti('fa.nii.gz', FA.astype("uint8"), img.affine)

RGB = color_fa(FA, tenfit.evecs)
save_nifti('rgb.nii.gz', RGB.astype("uint8"), img.affine)

# cfa = RGB[:, :, 40:41]
# cfa /= cfa.max()
# tensor_actor = actor.tensor_slicer(evals, evecs, scalar_colors=cfa, sphere=sphere, scale=0.3)
# ren.add(tensor_actor)
# if interactive:
#     window.show(ren)

In [8]:
img =("block_I35/wm_bin.nii.gz")
imgm = nib.load(img)
seed_mask = imgm.get_data()
seeds = random_seeds_from_mask(seed_mask, seeds_count=1)

img=("block_I35/dti_las_fa.nii")
imgm = nib.load(img)
fa = imgm.get_data()
tissue_classifier = ThresholdTissueClassifier(fa, 0.5)

In [20]:
data_small=data[40:80, 28:68, 28:68]
mask_small=seed_mask[40:80, 28:68, 28:68]

In [41]:
l_values = np.sqrt(gtab.bvals * 0.01506)
tmp=np.tile(l_values, (3,1))
gradsT = gtab.bvecs.T
b_vector = gradsT * tmp
b_vector = b_vector.T
gqi_vector = np.real(np.sinc(np.dot(b_vector, sphere.vertices.T)* 0.3/np.pi))
odfs = np.dot(data, gqi_vector)

In [42]:
#min/max normalization
ijk = np.ascontiguousarray(np.array(np.nonzero(seed_mask)).T)
shape = data.shape[:-1]
odfs_norm = np.zeros ((shape + (len(sphere.vertices),)))

for (k, center) in enumerate(ijk):
    m = odfs[tuple(center.astype(np.int))].copy()
    m = m - (np.abs(m).min())
    m = m / (np.abs(m).max())
    odfs_norm[tuple(center.astype(np.int))] = m

In [ ]:
npeaks = 3
relative_peak_threshold=.5
min_separation_angle=25
shape=mask.shape    
peak_dirs = np.zeros ((shape + (npeaks, 3)))
peak_values = np.zeros ((shape + (npeaks,)))
peak_indices = np.zeros((shape + (npeaks,)), dtype='int')

odfs_norm =odfs_gqi.astype(float)

for idx in ndindex(shape):
    if not mask[idx]:
        continue
    direction, pk, indices = peak_directions(odfs_norm[idx], sphere642, relative_peak_threshold, min_separation_angle)
    n = min(npeaks, pk.shape[0])
    
    peak_dirs[idx][:n] = direction[:n]
    peak_values[idx][:n] = pk[:n]
    peak_indices[idx][:n] = indices[:n]
    nib.save(nib.Nifti1Image(reshape_peaks_for_visualization(peak_dirs),
                                     img.affine),os.path.join(sub, 'peaks)gqi.nii.gz'))

In [43]:
nib.save(nib.Nifti1Image(odfs_norm.astype(np.float32),
                                  imgm.affine, imgm.header), 'block_I35/odfs_gqi_l03.nii.gz')


In [36]:
interactive = True
r = window.Renderer()
gqi_odfs_actor = actor.odf_slicer(odfs_norm, sphere=sphere, scale=0.6, norm=True, colormap= 'jet')
gqi_odfs_actor.display_extent(60, 60, 0, 96, 0, 96)                                    
r.set_camera(position=(10, 0, 0))
r.add(gqi_odfs_actor)
if interactive:
    window.show(r)

In [ ]:
img =("block_I35/wm_bin.nii.gz")
imgm = nib.load(img)
seed_mask = imgm.get_data()
seeds = random_seeds_from_mask(seed_mask, seeds_count=5)

img=("block_I35/dti_las_fa.nii")
imgm = nib.load(img)
fa = imgm.get_data()
tissue_classifier = ThresholdTissueClassifier(fa, 0.5)
pmf = odfs_norm.clip(min=0)
prob_dg = ProbabilisticDirectionGetter.from_pmf(pmf, max_angle=20, sphere=sphere, pmf_threshold=0.5)

all_streamlines = LocalTracking(prob_dg, tissue_classifier,
                                               seeds, 
                                               affine=np.eye(4), step_size=0.175,
                                               return_all=True)

streamlines = Streamlines(all_streamlines)
save_trk("block_I35/gqi_prob_npv5_20_0175_pmf05_l03.trk",
         streamlines,
         imgm.affine,
         shape=imgm.shape[:3], vox_size=imgm.header.get_zooms()[:3])